# **Tarea 3: Estadística descriptiva, Pandas y Spark**

## Alumno: José de Jesús Hernández Higuera

### Matrícula: 224470489

## **Proyecto hecho con Spark**

**Nota:** para una mejor visualización de los gráficos, tablas y resultados en general, es mejor abrir el notebook en Databricks en el enlace que se deja a continuación.

https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/2137159849679552/3977006123814776/8541298474673569/latest.html

### Preprocesamiento de los datos

*Este notebook estará desarrollado con Databricks, entonces, comenzaremos por subir los archivos de tipo csv y ver el directorio correspondiente a las tablas. Luego. leeremos los archivos para ir construyendo los Dataframes de Spark.*

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/2015-1.csv', name='2015-1.csv', size=56228885, modificationTime=1729890253000),
 FileInfo(path='dbfs:/FileStore/tables/2015.csv', name='2015.csv', size=56228885, modificationTime=1729761069000),
 FileInfo(path='dbfs:/FileStore/tables/2016.csv', name='2016.csv', size=68634144, modificationTime=1729914564000),
 FileInfo(path='dbfs:/FileStore/tables/2016_programa.csv', name='2016_programa.csv', size=17930526, modificationTime=1729914550000),
 FileInfo(path='dbfs:/FileStore/tables/2017.csv', name='2017.csv', size=13062694, modificationTime=1729915270000),
 FileInfo(path='dbfs:/FileStore/tables/2018.csv', name='2018.csv', size=12636453, modificationTime=1729915270000),
 FileInfo(path='dbfs:/FileStore/tables/2019.csv', name='2019.csv', size=20438624, modificationTime=1729915282000),
 FileInfo(path='dbfs:/FileStore/tables/2020.csv', name='2020.csv', size=24301567, modificationTime=1729915284000),
 FileInfo(path='dbfs:/FileStore/tables/LifeExpecta

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Out[3]: DataFrame[key: string, value: string]

In [0]:
capturas_2015 = spark.read.csv('dbfs:/FileStore/tables/2015.csv', header=True, inferSchema=True)
capturas_2016 = spark.read.csv('dbfs:/FileStore/tables/2016.csv', header=True, inferSchema=True)
capturas_programa = spark.read.csv('dbfs:/FileStore/tables/2016_programa.csv', header=True, inferSchema=True)
capturas_2017 = spark.read.csv('dbfs:/FileStore/tables/2017.csv', header=True, inferSchema=True)
capturas_2018 = spark.read.csv('dbfs:/FileStore/tables/2018.csv', header=True, inferSchema=True)
capturas_2019 = spark.read.csv('dbfs:/FileStore/tables/2019.csv', header=True, inferSchema=True)
capturas_2020 = spark.read.csv('dbfs:/FileStore/tables/2020.csv', header=True, inferSchema=True)

*Ahora procederemos a hacer algunas modificaciones para hacer más cómodo el trabajo.*

In [0]:
# El dataframe del 2016 y el que se llama programa tienen fechas del 2016, de modo que es conveniente unirlos en un sólo dataframe
cap_2016 = capturas_2016.union(capturas_programa)

# Cambiemos la columna llamada 'fecha' por una llamada 'Fecha'
capturas_2018 = capturas_2018.withColumnRenamed('fecha', 'Fecha')

In [0]:
# Ahora, convertiremos la columna 'Fecha' en un formato timestamp (intenté un ciclo for, pero no pareció funcionar)
# Adicionalmente, guardaremos los Dataframes en memoria para poder usarlos más adelante (excepto el 2019)
capturas_2015 = capturas_2015.withColumn('Fecha', to_timestamp('Fecha', 'dd/MM/yyyy')).cache()
capturas_2016 = cap_2016.withColumn('Fecha', to_timestamp('Fecha', 'dd/MM/yyyy')).cache()
capturas_2017 = capturas_2017.withColumn('Fecha', to_timestamp('Fecha', 'dd/MM/yyyy')).cache()
capturas_2018 = capturas_2018.withColumn('Fecha', to_timestamp('fecha', 'dd/MM/yyyy')).cache()
capturas_2020 = capturas_2020.withColumn('Fecha', to_timestamp('Fecha', 'dd/MM/yyyy')).cache()

*En la celda anterior no se guarda el año 2019 porque el resultado de la conversión hace que la columna 'Fecha' tenga valores nulos 'null' . Esto, como se muestra a continuación, es debido a que al menos una de las líneas de esta columna están vacías o tienen un formato inadecuado. De modo que antes de proceder con la conversión se debe limpiar esta columna.*

In [0]:
# En estas líneas descubriremos el problema que no permiten la conversión de la columna 'Fecha'
df_temp = capturas_2019.withColumn("Fecha_temp", to_timestamp("Fecha", "dd/MM/yyyy"))
df_temp.filter(col("Fecha_temp").isNull()).select("Fecha").show(truncate=False)
# Después, procedemos a realizar la limpieza necesaria
capturas_2019 = capturas_2019.withColumn("Fecha", trim(col("Fecha")))
# Finalmente, tenemos ahora sí la conversión adecuada y la guardaremos en memoria
capturas_2019 = capturas_2019.withColumn("Fecha", to_timestamp("Fecha", "dd/MM/yyyy")).cache()

+-----+
|Fecha|
+-----+
+-----+



In [0]:
# Comprobemos si la transformación ha resultado
capturas_2019.show(5)

+-------------------+--------------------+--------+---------+--------------------+---------+----------------+-------+----------+-------------+-----------------+-----------------+-----------------+---------------+--------------+-----------------+---------------+--------------+-----------------+---------------+--------------+-----------------+---------------+--------------+---+------+
|              Fecha|           Ubicacion| Latitud| Longitud|           Productor|Municipio|       Localidad|Cultivo|Superficie|NoTrampasInst|Trampas_revisadas|DiaphorinaXTrampa|Fenologia_Brote_1|Adultos_Brote_1|Ninfas_Brote_1|Fenologia_Brote_2|Adultos_Brote_2|Ninfas_Brote_2|Fenologia_Brote_3|Adultos_Brote_3|Ninfas_Brote_3|Fenologia_Brote_4|Adultos_Brote_4|Ninfas_Brote_4|Mes|Semana|
+-------------------+--------------------+--------+---------+--------------------+---------+----------------+-------+----------+-------------+-----------------+-----------------+-----------------+---------------+--------------+-

*A continuación, veremos que el DataFrame del 2016 tiene fechas que pertenecen al 2015. Para mostrarlo primero crearemos una vista temporal del DataFrame del 2016 en SQL y, posteriormente haremos el filtrado usando la cláusula 'where'.*

In [0]:
capturas_2016.createOrReplaceTempView('cap_2016')

In [0]:
%sql
select * from `cap_2016` where `Fecha` like '%2015%' limit 10

Fecha,Latitud,Longitud,Ubicacion,Municipio,Comunidad,Productor,Cultivo,Fenologia,Superficie_ha,No_Trampas_Instaladas,No_Trampas_Revisadas,No_DiaphorinasXtrampa,Mes,Semana
2015-01-09T00:00:00.000+0000,23.93176,-99.17629,177_THUE05-28-041-0068T01,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),90.0,0,0,0,1,2
2015-01-02T00:00:00.000+0000,23.93176,-99.17629,177_THUE05-28-041-0068T01,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,FRUCTIFICACIÓN,90.0,0,1,0,1,1
2015-01-16T00:00:00.000+0000,23.93318,-99.17713,177_THUE05-28-041-0068T02,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,3
2015-01-09T00:00:00.000+0000,23.93318,-99.17713,177_THUE05-28-041-0068T02,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,2
2015-01-02T00:00:00.000+0000,23.93318,-99.17713,177_THUE05-28-041-0068T02,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,FRUCTIFICACIÓN,0.0,0,1,0,1,1
2015-01-16T00:00:00.000+0000,23.93459,-99.17802,177_THUE05-28-041-0068T03,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,3
2015-01-09T00:00:00.000+0000,23.93459,-99.17802,177_THUE05-28-041-0068T03,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,2
2015-01-02T00:00:00.000+0000,23.93459,-99.17802,177_THUE05-28-041-0068T03,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,FRUCTIFICACIÓN,0.0,0,1,0,1,1
2015-01-09T00:00:00.000+0000,23.936,-99.17883,177_THUE05-28-041-0068T04,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,2
2015-01-16T00:00:00.000+0000,23.936,-99.17883,177_THUE05-28-041-0068T04,VICTORIA,Victoria,CERVERA_HERRERA_JAVIER,NARANJA,DESARROLLO (crecimiento),0.0,0,0,0,1,3


*Entonces, será conveniente hacer las modificaciones necesarias para que los DataFrames del 2015 y del 2016 tengan las fechas correspondientes al año que representan. En este caso, lo que se hará será separar las filas con fechas del 2015 en un nuevo DataFrame, unirlo con el del 2015 y eliminar estas estradas de la tabla del 2016.*

In [0]:
# Separemos los datos correspondientes al 2015
sub_2015 = capturas_2016.filter(year('Fecha') == 2015)
# Vamos a unirlos al DataFrame correspondiente
capturas_2015 = capturas_2015.union(sub_2015)
# Finalmente, quitemos las filas del 2015 del dataframe del 2016
capturas_2016 = capturas_2016.filter(year('Fecha') != 2015)

In [0]:
# Eliminemos los posibles valores nulos que pudiesen existir
capturas_2015 = capturas_2015.dropna()

In [0]:
# Por último, guardemos estas últimas correcciones en memoria
capturas_2015.cache()
capturas_2016.cache()

Out[13]: DataFrame[Fecha: timestamp, Latitud: double, Longitud: double, Ubicacion: string, Municipio: string, Comunidad: string, Productor: string, Cultivo: string, Fenologia: string, Superficie_ha: double, No_Trampas_Instaladas: int, No_Trampas_Revisadas: int, No_DiaphorinasXtrampa: int, Mes: int, Semana: int]

### 1) Hacer una gráfica del promedio de Diaphorinas por trampa para cada mes de los años 2015, 2017 y 2020. Es decir una gráfica por cada año donde el eje horizontal esté el mes y el vertical el promedio de diaphorinas por trampa.

In [0]:
#capturas_2015 = capturas_2015.withColumn("Mes", trunc("Fecha", "month")).cache()
#capturas_2017 = capturas_2017.withColumn("Mes", trunc("Fecha", "month")).cache()
#capturas_2020 = capturas_2020.withColumn("Mes", trunc("Fecha", "month")).cache()

In [0]:
prom_2015 = capturas_2015.groupBy("Mes").agg({"No_DiaphorinasXtrampa": "avg"}).withColumnRenamed("avg(No_DiaphorinasXtrampa)", "Promedio 2015").cache()
prom_2017 = capturas_2017.groupBy("Mes").agg({"No_DiaphorinasXtrampa": "avg"}).withColumnRenamed("avg(No_DiaphorinasXtrampa)", "Promedio 2017").cache()
prom_2020 = capturas_2020.groupBy("Mes").agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Promedio 2020").cache()

In [0]:
resultado = prom_2015.join(prom_2017, "Mes", "outer").join(prom_2020, "Mes", "outer").orderBy('Mes')

In [0]:
display(resultado)

Mes,Promedio 2015,Promedio 2017,Promedio 2020
1,0.039572549674403074,0.14972527472527472,0.37856461359058624
2,0.11651014455591525,0.27774725274725276,0.7503039144177
3,0.1608653846153846,0.566025641025641,0.5028490028490028
4,0.10546153846153847,0.28214285714285714,0.44607519788918204
5,0.03711419505785071,0.5130494505494505,0.24975905273303042
6,0.0330448717948718,0.8178571428571428,0.21221959858323494
7,0.039219707262054294,0.30354107648725215,0.30578512396694213
8,0.1148076923076923,0.5685606060606061,0.28952524491333836
9,0.15618589743589745,0.6445054945054945,0.19150529581683962
10,0.1755189729254025,0.5184065934065935,0.18981921979067554


Databricks visualization. Run in Databricks to view.

In [0]:
# Por último, liberemos la memoria de los dataframes guardados en este ejercicio
prom_2015.unpersist()
prom_2017.unpersist()
prom_2020.unpersist()

Out[18]: DataFrame[Mes: int, Promedio 2020: double]

### 2) Hacer una gráfica del promedio de Diaphorinas por trampa para cada mes del año 2020, pero sólo de un municipio.

In [0]:
# Veamos primero las columnas de este dataframe para saber cuál de ellas corresponde al municipio
print(capturas_2020.columns)

['Fecha', 'Ubicacion', 'Latitud', 'Longitud', 'Productor', 'Municipio', 'Localidad', 'Cultivo', 'Variedad', 'noProductores', 'Superficie', 'NoTrampasInst', 'Trampas_revisadas', 'DiaphorinaXTrampa', 'Observacion', 'Incidencia_Psilido', 'Fenologia_Brote_1', 'Adultos_Brote_1', 'Ninfas_Brote_1', 'Fenologia_Brote_2', 'Adultos_Brote_2', 'Ninfas_Brote_2', 'Fenologia_Brote_3', 'Adultos_Brote_3', 'Ninfas_Brote_3', 'Fenologia_Brote_4', 'Adultos_Brote_4', 'Ninfas_Brote_4', 'Mes', 'Semana', 'Tecnico']


In [0]:
# Ahora podemos ver cuántos municipios distintos hay
capturas_2020.select('Municipio').distinct().show()

+---------+
|Municipio|
+---------+
|  PADILLA|
| VICTORIA|
|  HIDALGO|
|   GUEMEZ|
|    LLERA|
+---------+



*A partir de lo anterior, podemos hacer una consulta para saber cuál es el promedio mensual de Diaphorinas por trampa en el 2020 en un municipio, por ejemplo, en Padilla. Adicionalmente, gracias a las herramientas de Databricks, podemos crear la gráfica correspondiente.*

In [0]:
padilla = capturas_2020.where(col('Municipio')=='PADILLA').groupBy('Mes').agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Padilla").orderBy('Mes').cache()
victoria = capturas_2020.where(col('Municipio')=='VICTORIA').groupBy('Mes').agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Victoria").orderBy('Mes').cache()
hidalgo = capturas_2020.where(col('Municipio')=='HIDALGO').groupBy('Mes').agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Hidalgo").orderBy('Mes').cache()
guemez = capturas_2020.where(col('Municipio')=='GUEMEZ').groupBy('Mes').agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Güemez").orderBy('Mes').cache()
llera = capturas_2020.where(col('Municipio')=='LLERA').groupBy('Mes').agg({"DiaphorinaXTrampa": "avg"}).withColumnRenamed("avg(DiaphorinaXTrampa)", "Llera").orderBy('Mes').cache()

In [0]:
resultado2 = padilla.join(victoria, "Mes", "outer").join(hidalgo, "Mes", "outer").join(guemez, "Mes", "outer").join(llera, "Mes", "outer").orderBy('Mes')

In [0]:
display(resultado2)

Mes,Padilla,Victoria,Hidalgo,Güemez,Llera
1,0.07737348389795065,0.9082240762812872,0.21842650103519667,0.4083449235048679,0.5755176613885505
2,0.3262964662689307,0.9063670411985019,0.7914746543778802,0.7857589006870705,1.3027210884353742
3,0.17546296296296296,0.55,0.6036866359447005,0.6303789338471419,0.7074468085106383
4,0.3051608077786088,0.9666666666666667,0.21076746849942726,0.513235294117647,0.38685015290519875
5,0.1129242819843342,0.2423076923076923,0.22863741339491916,0.29549193813754526,0.3208269525267994
6,0.14100185528756956,0.20192307692307693,0.2496551724137931,0.23044798785117693,0.24765625
7,0.8219616204690832,0.07981220657276995,0.3608815426997245,0.007462686567164179,null
8,0.13586606567933032,0.7457180500658761,0.25504322766570603,0.1606822262118492,0.4346895074946467
9,0.050821551394726786,0.3016759776536313,0.17343173431734318,0.20945319740500462,0.349812734082397
10,0.015917602996254682,0.13690476190476192,0.13294117647058823,0.273210378243201,0.33261026753434564


Databricks visualization. Run in Databricks to view.

In [0]:
# Finalmente, limpiemos la memoria de los dataframes guardados en esta sección
padilla.unpersist()
victoria.unpersist()
hidalgo.unpersist()
guemez.unpersist()
llera.unpersist()

Out[24]: DataFrame[Mes: int, Llera: double]

### 3) Hacer una tabla donde la primera columna sea el mes, la segunda el índice Id (por trampa) y la tercera el índice Cx para el año 2015

In [0]:
# Primero obtengamos la varianza y la media muestrales de los datos del 2015
tabla = capturas_2015.groupBy("Mes").agg(avg("No_DiaphorinasXtrampa").alias("Promedio"), variance("No_DiaphorinasXtrampa").alias("Varianza"), sum("No_DiaphorinasXtrampa").alias("Suma"))
# Posteriormente, construyamos las columnas necesarias para mostrar los índices necesarios
tabla = tabla.withColumn("Índice de dispersión", col("Varianza") / col("Promedio")).orderBy("Mes").withColumn("Índice de Green", ((col("Varianza") / col("Promedio")) - 1) / (col('Suma') - 1)).orderBy("Mes")

# Finalmente, seleccionemos las columnas necesarias
tabla = tabla.select('Mes', 'Índice de dispersión', 'Índice de Green')

In [0]:
display(tabla)

Mes,Índice de dispersión,Índice de Green
1,2.551215747169398,0.0010916366975154102
2,3.39969235851098,6.603446225952064E-4
3,4.7408605847732295,7.454883588627401E-4
4,2.773530073682528,4.3130595177104283E-4
5,2.3601980647447034,0.001175624947921092
6,5.1048491040384505,0.00398529039227034
7,4.023628531648487,0.001977520295388154
8,5.398486497468066,0.0012282844170533555
9,8.134860798754211,0.0014644623971170384
10,8.646265068302219,0.0011277677091891178


Databricks visualization. Run in Databricks to view.

### 4) Hacer una gráfica del índice Id (por trampa) como función del mes que abarque desde 2015 hasta 2020

*Como podremos ver, el Dataframe del 2018 no tiene una columna que señale el mes como sí lo tienen el resto de Dataframes. En consecuencia, será necesario crear una columna con este nombre.*

In [0]:
capturas_2018.columns

Out[28]: ['Fecha',
 'ano',
 'semana',
 'name_thue',
 'latitud',
 'longitud',
 'fenologia',
 'Revisada',
 'captura',
 'campo',
 'superficie']

In [0]:
# Creamos la columna 'Mes' y la guardamos en memoria
capturas_2018 = capturas_2018.withColumn('Mes', month('Fecha'))
capturas_2018.cache()

Out[32]: DataFrame[Fecha: timestamp, ano: int, semana: int, name_thue: string, latitud: double, longitud: double, fenologia: string, Revisada: string, captura: int, campo: string, superficie: double, Mes: int]

*Ahora ya es posible extraer el índice de dispersión de cada uno de los meses de cada año. Posteriormente, se juntarán los dataframes, se harán las selecciones de las columnas correspondientes y podremos hacer la visualización de la gráfica.*

In [0]:
Id_2015 = capturas_2015.groupBy("Mes").agg(avg("No_DiaphorinasXtrampa").alias("Promedio_15"), variance("No_DiaphorinasXtrampa").alias("Varianza_15"))
Id_2015 = Id_2015.withColumn("Id 2015", col("Varianza_15") / col("Promedio_15")).orderBy("Mes")

Id_2016 = capturas_2016.groupBy("Mes").agg(avg("No_DiaphorinasXtrampa").alias("Promedio_16"), variance("No_DiaphorinasXtrampa").alias("Varianza_16"))
Id_2016 = Id_2016.withColumn("Id 2016", col("Varianza_16") / col("Promedio_16")).orderBy("Mes")

Id_2017 = capturas_2017.groupBy("Mes").agg(avg("No_DiaphorinasXtrampa").alias("Promedio_17"), variance("No_DiaphorinasXtrampa").alias("Varianza_17"))
Id_2017 = Id_2017.withColumn("Id 2017", col("Varianza_17") / col("Promedio_17")).orderBy("Mes")

# Para 2018 usaremos la columna 'captura'
Id_2018 = capturas_2018.groupBy("Mes").agg(avg("captura").alias("Promedio_18"), variance("captura").alias("Varianza_18"))
Id_2018 = Id_2018.withColumn("Id 2018", col("Varianza_18") / col("Promedio_18")).orderBy("Mes")

# En 2019 y 2020 la columna se llama 'DiaphorinaXTrampa'
Id_2019 = capturas_2019.groupBy("Mes").agg(avg("DiaphorinaXTrampa").alias("Promedio_19"), variance("DiaphorinaXTrampa").alias("Varianza_19"))
Id_2019 = Id_2019.withColumn("Id 2019", col("Varianza_19") / col("Promedio_19")).orderBy("Mes")

Id_2020 = capturas_2020.groupBy("Mes").agg(avg("DiaphorinaXTrampa").alias("Promedio_20"), variance("DiaphorinaXTrampa").alias("Varianza_20"))
Id_2020 = Id_2020.withColumn("Id 2020", col("Varianza_20") / col("Promedio_20")).orderBy("Mes")

In [0]:
resultado4 = Id_2015.join(Id_2016, "Mes", "outer").join(Id_2017, "Mes", "outer").join(Id_2018, "Mes", "outer").join(Id_2019, "Mes", "outer").join(Id_2020, "Mes", "outer").orderBy('Mes')
resultado4 = resultado4.select('Mes', 'Id 2015', 'Id 2016', 'Id 2017', 'Id 2018', 'Id 2018', 'Id 2019', 'Id 2020')
display(resultado4)

Mes,Id 2015,Id 2016,Id 2017,Id 2018,Id 2018,Id 2019,Id 2020
1,2.551215747169398,7.510767722614624,3.4787533625277693,2.4007627265598126,2.4007627265598126,3.257349824609542,5.60942853430792
2,3.39969235851098,9.761346381080557,6.544121002262633,2.2882601522045634,2.2882601522045634,null,5.005975412048133
3,4.7408605847732295,11.717068068451315,7.88591799783373,11.59480626519134,11.59480626519134,3.0002524309698932,3.713701970968405
4,2.773530073682528,4.223621957089599,4.195551273735267,2.0661266285274804,2.0661266285274804,2.4795191999802952,5.9020330278227675
5,2.3601980647447034,3.976548211143057,9.100248776394208,7.616306463325428,7.616306463325428,2.7139404189561036,2.537800946583641
6,5.1048491040384505,3.01989598314411,36.16057318684908,2.6649475511585554,2.6649475511585554,2.6024736937211608,2.0314738079230574
7,4.023628531648487,3.96543915917634,6.128964939665585,2.815587754185893,2.815587754185893,3.5012842232488888,27.744613930305537
8,5.398486497468066,4.662794002169112,12.437947009977057,2.4482225997222407,2.4482225997222407,2.078173804633263,4.162429533030435
9,8.134860798754211,5.79241320153555,6.549488789704044,3.2582447591983787,3.2582447591983787,3.742554714617905,2.4970300083565733
10,8.646265068302219,5.216585685381131,13.786083813786389,4.267784767801854,4.267784767801854,3.8189421527798935,2.3392810562149404


Databricks visualization. Run in Databricks to view.

*Notemos que, para el mes de febrero, el índice de dispersión de 2019 tiene un valor nulo. Esto se debe a que, como podremos ver en la siguiente celda, no hay valores distintos de 0 en la columna 'DiaphorinaXTrampa' para el mes de febrero. Esto implica que el promedio en este mes es 0 y, en consecuencia, el índice está indefinido.*

In [0]:
capturas_2019.select('Mes', 'DiaphorinaXTrampa').where((col('DiaphorinaXTrampa')!=0) & (col('Mes')==2)).show()

+---+-----------------+
|Mes|DiaphorinaXTrampa|
+---+-----------------+
+---+-----------------+



### 5) Hacer una gráfica del centro de masa de las diaphorinas como función del mes que abarque de 2017 a 2020.  Nota: CM=pos*Diaph/Ntrampas, donde CM se refiere al centro de masa, pos es la posición de la trampa, Diaph es el promedio de diaphorinas en la trampa y Ntrampas es el número de trampas que se consideran en el cálculo.

In [0]:
CM_2017 = capturas_2017.groupBy('Mes').agg(avg('Latitud').alias('Latitud'), avg('Longitud').alias('Longitud'), avg('No_DiaphorinasXtrampa').alias('Diaph'), sum('No_Trampas_Revisadas').alias('Ntrampas')).withColumn('CM latitud 17', (col('Latitud') * col('Diaph')) / col('Ntrampas')).withColumn('CM longitud 17', (col('Longitud') * col('Diaph')) / col('Ntrampas')).orderBy('Mes').cache()

# Contamos sólo aquellas trampas que fueron revisadas, es decir, aquellas con un 'Si' en la columna 'revisada'
CM_2018 = capturas_2018.groupBy('Mes').agg(avg('Latitud').alias('Latitud'), avg('Longitud').alias('Longitud'), avg('captura').alias('Diaph'), count(when(col('revisada') == 'Si', True)).alias('Ntrampas')).withColumn('CM latitud 18', (col('Latitud') * col('Diaph')) / col('Ntrampas')).withColumn('CM longitud 18', (col('Longitud') * col('Diaph')) / col('Ntrampas')).orderBy('Mes').cache()

CM_2019 = capturas_2019.groupBy('Mes').agg(avg('Latitud').alias('Latitud'), avg('Longitud').alias('Longitud'), avg('DiaphorinaXtrampa').alias('Diaph'), sum('Trampas_revisadas').alias('Ntrampas')).withColumn('CM latitud 19', (col('Latitud') * col('Diaph')) / col('Ntrampas')).withColumn('CM longitud 19', (col('Longitud') * col('Diaph')) / col('Ntrampas')).orderBy('Mes').cache()

CM_2020 = capturas_2020.groupBy('Mes').agg(avg('Latitud').alias('Latitud'), avg('Longitud').alias('Longitud'), avg('DiaphorinaXtrampa').alias('Diaph'), sum('Trampas_revisadas').alias('Ntrampas')).withColumn('CM latitud 20', (col('Latitud') * col('Diaph')) / col('Ntrampas')).withColumn('CM longitud 20', (col('Longitud') * col('Diaph')) / col('Ntrampas')).orderBy('Mes').cache()

In [0]:
resultado5 = CM_2017.join(CM_2018, "Mes", "outer").join(CM_2019, "Mes", "outer").join(CM_2020, "Mes", "outer").orderBy('Mes')
resultado5 = resultado5.select('Mes', 'CM latitud 17', 'CM longitud 17', 'CM latitud 18', 'CM longitud 18', 'CM latitud 19', 'CM longitud 19', 'CM latitud 20', 'CM longitud 20')

display(resultado5)

Mes,CM latitud 17,CM longitud 17,CM latitud 18,CM longitud 18,CM latitud 19,CM longitud 19,CM latitud 20,CM longitud 20
1,9.8746116598606E-4,-0.004077278838841787,1.9395035264208074E-4,-8.026657959314409E-4,5.249111457793091E-4,-0.002177944620347419,9.585824713405024E-4,-0.0039769177032431486
2,9.161445677120042E-4,-0.0037828088718019916,3.8144575878109417E-4,-0.0015822091417411676,null,null,0.0022188178908565132,-0.009202429865126928
3,0.0012446839373618825,-0.00513936512509245,6.508876840596804E-4,-0.0027024969878041713,3.095120975624584E-4,-0.0012838370146007068,0.0015706558112850062,-0.00651235833528963
4,9.305155405161686E-4,-0.0038421473706982804,6.391053032213386E-4,-0.0026509706048862707,3.656816330528961E-4,-0.0015174457321023805,0.0018291019713306737,-0.007589504174918245
5,0.0017051708591498465,-0.007040739727398535,8.849338109985612E-4,-0.0036706523526283887,7.063623273612865E-4,-0.00292995771319217,8.210858959697329E-4,-0.0034073878026730977
6,0.003024787188171669,-0.012489420510066674,6.415295207261686E-4,-0.002661026182197523,5.654546056325249E-4,-0.002345471676219059,8.353925169816414E-4,-0.003465278985365338
7,0.0014637855866037274,-0.006044054379662662,4.492946175437249E-4,-0.0018619375800434381,2.6930479709918467E-4,-0.0011171419458886138,0.0053502777851914905,-0.022197214518016244
8,0.0013319974266858584,-0.005499798575318649,3.7695518191047724E-4,-0.0015650494290645155,3.7785217698954387E-4,-0.0015673079013884875,0.0012237677114313047,-0.005075885127131947
9,0.0022031076955645835,-0.009096622721796895,6.476276674017356E-4,-0.0026863195592100628,9.283092170825037E-4,-0.003850904100373539,5.013259853542632E-4,-0.002078011865403687
10,0.002560741860628963,-0.01057330258823909,0.0017991545904027665,-0.007462781814597759,6.554228208366947E-4,-0.002718602992567207,5.413813958710647E-4,-0.0022453043302629225


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Por último, liberemos las variables guardadas en la memoria
CM_2017.unpersist()
CM_2018.unpersist()
CM_2019.unpersist()
CM_2020.unpersist()

Out[63]: DataFrame[Mes: int, Latitud: double, Longitud: double, Diaph: double, Ntrampas: bigint, CM latitud 20: double, CM longitud 20: double]